# WorldScreener Demo Notebook

This notebook demonstrates the capabilities of the WorldScreener package with various examples of screening, analyzing, and visualizing value stocks.

## Setup and Configuration

In [ ]:
import os
import sys
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML, display, Markdown
from tabulate import tabulate

# Add source directory to path
sys.path.insert(0, os.path.join(os.path.dirname(os.path.dirname('__file__')), '..'))

# Import WorldScreener modules
from src import create_screener, setup_logging, load_config
from src.utils import create_output_dir, format_currency, format_percentage

# Set up output directory
output_dir = os.path.join(os.path.dirname(os.path.dirname('__file__')), 'output')
create_output_dir(output_dir)
print(f"Demo output will be saved to: {output_dir}")

# Configure visualization style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.facecolor'] = 'white'

## Initialize WorldScreener Components

We'll initialize the WorldScreener components with the option to use either mock data or real data from Yahoo Finance.

In [ ]:
# Set to True to use real data (may hit API rate limits)
use_real_data = False

print("Initializing WorldScreener components...")

# Load configuration
config = load_config()

# Create WorldScreener instance
if use_real_data:
    print("Using REAL data from Yahoo Finance (may hit API rate limits)")
else:
    print("Using MOCK data for demonstration (avoiding Yahoo Finance API rate limits)")

# Create a screener instance
screener = create_screener(use_mock_data=not use_real_data)

# Extract components from screener
data_provider = screener.data_provider
analyzer = screener.analyzer
visualizer = screener.visualizer
report_generator = screener.report_generator

## Demonstration 1: Finding Value Stocks in Spain

This demonstration shows how to screen Spanish stocks for value investing opportunities using default criteria.

In [ ]:
# Screen Spanish stocks with default criteria
print("Screening Spanish stocks with default criteria...")
spanish_stocks = screener.screen_stocks(region='spain', limit=15)

if spanish_stocks.empty:
    print("No Spanish stocks found meeting the criteria.")
else:
    # Display the results
    print(f"\nFound {len(spanish_stocks)} Spanish value stocks:")
    
    # Format the dataframe for display
    display_df = spanish_stocks[['ticker', 'name', 'sector', 'pe_ratio', 'dividend_yield', 'value_score']].copy()
    
    # Display as a styled dataframe
    display(display_df.style
           .format({'pe_ratio': '{:.2f}', 'dividend_yield': '{:.2f}%', 'value_score': '{:.2f}'})
           .background_gradient(cmap='YlGn', subset=['value_score'])
           .set_caption('Spanish Value Stocks'))

### Sector Breakdown Visualization

Let's visualize the sector distribution of the screened Spanish stocks.

In [ ]:
if not spanish_stocks.empty:
    # Create and display sector breakdown chart
    fig = visualizer.create_sector_breakdown_chart(spanish_stocks, title="Sector Breakdown of Spanish Value Stocks")
    plt.show()
    
    # Save the chart
    chart_file = os.path.join(output_dir, 'spanish_stocks_by_sector.png')
    plt.savefig(chart_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"Chart saved to {chart_file}")

### Detailed Analysis of Top Stock

Now let's perform a detailed analysis of the top Spanish stock based on value score.

In [ ]:
if not spanish_stocks.empty:
    # Get the top stock
    top_stock = spanish_stocks.iloc[0]['ticker']
    print(f"Analyzing top stock: {top_stock}...")
    
    # Perform detailed analysis
    analysis = analyzer.analyze_stock(top_stock)
    
    # Display key metrics
    if analysis:
        metrics = {
            'Metric': ['Company Name', 'Sector', 'Price', 'P/E Ratio', 'P/B Ratio', 'Dividend Yield', 'ROE', 'Debt/Equity'],
            'Value': [
                analysis['name'],
                analysis['sector'],
                f"{analysis['price']:.2f} {analysis['currency']}",
                f"{analysis['pe_ratio']:.2f}",
                f"{analysis['pb_ratio']:.2f}",
                f"{analysis['dividend_yield']:.2f}%",
                f"{analysis['roe']:.2f}%",
                f"{analysis['debt_to_equity']:.2f}"
            ]
        }
        
        metrics_df = pd.DataFrame(metrics)
        display(metrics_df.style.set_caption(f"Key Metrics for {top_stock}"))
        
        # Generate and save analysis report
        report_file = os.path.join(output_dir, f"{top_stock}_analysis.html")
        report_generator.generate_stock_analysis_report(analysis, output_file=report_file)
        print(f"Analysis report saved to {report_file}")
        
        # Display a link to open the HTML report
        display(HTML(f'<a href="{report_file}" target="_blank">Open {top_stock} Analysis Report</a>'))

## Demonstration 2: Multi-Region Screening with Custom Criteria

This demonstration shows how to screen stocks from multiple regions using custom criteria.

In [ ]:
# Define custom screening criteria
custom_criteria = {
    'max_pe_ratio': 12.0,
    'min_dividend_yield': 4.0,
    'max_pb_ratio': 1.2,
    'min_roe': 12.0,
    'max_debt_to_equity': 0.8
}

# Display custom criteria
print("Custom screening criteria:")
for key, value in custom_criteria.items():
    print(f"- {key}: {value}")

# Function to screen and display results for a region
def screen_region(region, criteria):
    print(f"\nScreening {region} stocks with custom criteria...")
    stocks = screener.screen_stocks(region=region, criteria=criteria, limit=10)
    
    if stocks.empty:
        print(f"No {region} stocks found meeting the criteria.")
        return None
    else:
        print(f"Found {len(stocks)} {region} stocks meeting the criteria:")
        display_df = stocks[['ticker', 'name', 'sector', 'country', 'pe_ratio', 'dividend_yield', 'value_score']].copy()
        display(display_df.style
               .format({'pe_ratio': '{:.2f}', 'dividend_yield': '{:.2f}%', 'value_score': '{:.2f}'})
               .background_gradient(cmap='YlGn', subset=['value_score'])
               .set_caption(f'{region.capitalize()} Value Stocks'))
        return stocks

# Screen Europe stocks
europe_stocks = screen_region('europe', custom_criteria)

# Screen US stocks
us_stocks = screen_region('us', custom_criteria)

### Comparison of Stocks Across Regions

If we found stocks in both regions, let's compare them.

In [ ]:
if europe_stocks is not None and us_stocks is not None and not europe_stocks.empty and not us_stocks.empty:
    # Combine stocks from both regions
    combined_stocks = pd.concat([europe_stocks, us_stocks])
    
    # Create comparison chart
    plt.figure(figsize=(14, 8))
    
    # Plot P/E ratio vs Dividend Yield with bubble size representing Value Score
    scatter = plt.scatter(combined_stocks['pe_ratio'], 
                         combined_stocks['dividend_yield'],
                         s=combined_stocks['value_score'] * 5,  # Size based on value score
                         c=combined_stocks['country'].astype('category').cat.codes,  # Color by country
                         alpha=0.7)
    
    # Add labels for each point
    for i, row in combined_stocks.iterrows():
        plt.annotate(row['ticker'], 
                    (row['pe_ratio'], row['dividend_yield']),
                    xytext=(5, 5),
                    textcoords='offset points')
    
    # Add legend for countries
    legend1 = plt.legend(scatter.legend_elements()[0], 
                         combined_stocks['country'].unique(),
                         title="Country",
                         loc="upper left")
    plt.gca().add_artist(legend1)
    
    # Add labels and title
    plt.xlabel('P/E Ratio')
    plt.ylabel('Dividend Yield (%)')
    plt.title('P/E Ratio vs Dividend Yield (bubble size = Value Score)')
    plt.grid(True, alpha=0.3)
    
    # Show the plot
    plt.tight_layout()
    plt.show()
else:
    print("\nInsufficient data to create comparison chart. Try relaxing the screening criteria.")

## Demonstration 3: Value Portfolio Creation and Analysis

This demonstration shows how to create and analyze a portfolio of value stocks from multiple regions.

In [ ]:
print("Creating a portfolio with top value stocks from each region...")

# Function to get top stocks from a region
def get_top_stocks(region, limit=5):
    stocks = screener.screen_stocks(region=region, limit=limit)
    if stocks.empty:
        print(f"No stocks found in {region} region meeting the criteria.")
        return None
    return stocks

# Get stocks from different regions
regions = ['europe', 'us']
portfolio_stocks = []

for region in regions:
    stocks = get_top_stocks(region)
    if stocks is not None:
        portfolio_stocks.append(stocks)

# Combine stocks into a portfolio
if portfolio_stocks:
    portfolio = pd.concat(portfolio_stocks)
    print(f"\nCreated portfolio with {len(portfolio)} stocks:")
    
    # Display portfolio
    display_df = portfolio[['ticker', 'name', 'sector', 'country', 'pe_ratio', 'dividend_yield', 'value_score']].copy()
    display(display_df.style
           .format({'pe_ratio': '{:.2f}', 'dividend_yield': '{:.2f}%', 'value_score': '{:.2f}'})
           .background_gradient(cmap='YlGn', subset=['value_score'])
           .set_caption('Value Portfolio'))
else:
    print("No stocks found for portfolio. Try relaxing the screening criteria.")

### Portfolio Allocation Visualization

Let's visualize the portfolio allocation by sector and country.

In [ ]:
if 'portfolio' in locals() and not portfolio.empty:
    # Create portfolio allocation chart
    fig = visualizer.create_portfolio_allocation_chart(portfolio, title="Portfolio Allocation by Sector and Country")
    plt.show()
    
    # Save the chart
    chart_file = os.path.join(output_dir, 'portfolio_allocation.png')
    plt.savefig(chart_file, dpi=300, bbox_inches='tight', facecolor='white')
    print(f"Chart saved to {chart_file}")
    
    # Perform detailed analysis on top stocks
    detailed_analysis = {}
    top_stocks = portfolio.head(3)['ticker'].tolist()
    print(f"\nPerforming detailed analysis on top stocks: {', '.join(top_stocks)}")
    
    for ticker in top_stocks:
        detailed_analysis[ticker] = analyzer.analyze_stock(ticker)
    
    # Generate portfolio report
    report_file = os.path.join(output_dir, 'portfolio_report.html')
    report_generator.generate_portfolio_report(portfolio, detailed_analysis, output_file=report_file)
    print(f"Portfolio report saved to {report_file}")
    
    # Display a link to open the HTML report
    display(HTML(f'<a href="{report_file}" target="_blank">Open Portfolio Report</a>'))

## Portfolio Performance Simulation

Let's create a simple simulation of how this portfolio might perform over time based on historical data.

In [ ]:
import numpy as np
from datetime import datetime, timedelta

if 'portfolio' in locals() and not portfolio.empty:
    # Create a simulation of portfolio performance
    print("Simulating portfolio performance over 3 years...")
    
    # Parameters for simulation
    num_days = 365 * 3  # 3 years
    start_date = datetime.now() - timedelta(days=num_days)
    dates = [start_date + timedelta(days=i) for i in range(num_days)]
    
    # Create random returns with realistic parameters
    np.random.seed(42)  # For reproducibility
    annual_return = 0.08  # 8% expected annual return for value stocks
    annual_volatility = 0.15  # 15% annual volatility
    daily_return = annual_return / 252  # Trading days in a year
    daily_volatility = annual_volatility / np.sqrt(252)
    
    # Generate random daily returns
    daily_returns = np.random.normal(daily_return, daily_volatility, num_days)
    
    # Calculate cumulative returns
    cumulative_returns = np.cumprod(1 + daily_returns)
    
    # Create a benchmark (e.g., S&P 500)
    benchmark_annual_return = 0.07  # 7% expected annual return
    benchmark_annual_volatility = 0.18  # 18% annual volatility
    benchmark_daily_return = benchmark_annual_return / 252
    benchmark_daily_volatility = benchmark_annual_volatility / np.sqrt(252)
    
    # Generate random daily returns for benchmark
    benchmark_daily_returns = np.random.normal(benchmark_daily_return, benchmark_daily_volatility, num_days)
    benchmark_cumulative_returns = np.cumprod(1 + benchmark_daily_returns)
    
    # Plot the performance
    plt.figure(figsize=(14, 8))
    plt.plot(dates, cumulative_returns, label='Value Portfolio', linewidth=2)
    plt.plot(dates, benchmark_cumulative_returns, label='Market Benchmark', linewidth=2, alpha=0.7)
    plt.xlabel('Date')
    plt.ylabel('Cumulative Return (starting at 1.0)')
    plt.title('Simulated Portfolio Performance vs Benchmark (3 Years)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Format x-axis dates
    plt.gcf().autofmt_xdate()
    
    # Add annotations
    final_portfolio_return = (cumulative_returns[-1] - 1) * 100
    final_benchmark_return = (benchmark_cumulative_returns[-1] - 1) * 100
    plt.annotate(f'Portfolio: +{final_portfolio_return:.2f}%', 
                xy=(dates[-1], cumulative_returns[-1]),
                xytext=(10, 10),
                textcoords='offset points',
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2'))
    
    plt.annotate(f'Benchmark: +{final_benchmark_return:.2f}%', 
                xy=(dates[-1], benchmark_cumulative_returns[-1]),
                xytext=(10, -20),
                textcoords='offset points',
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.2'))
    
    plt.tight_layout()
    plt.show()
    
    # Calculate and display performance metrics
    metrics = {
        'Metric': ['Total Return', 'Annualized Return', 'Maximum Drawdown', 'Sharpe Ratio'],
        'Portfolio': [
            f"{final_portfolio_return:.2f}%",
            f"{annual_return * 100:.2f}%",
            f"{(1 - np.min(cumulative_returns) / np.max(cumulative_returns[0:np.argmin(cumulative_returns)])) * 100:.2f}%",
            f"{annual_return / annual_volatility:.2f}"
        ],
        'Benchmark': [
            f"{final_benchmark_return:.2f}%",
            f"{benchmark_annual_return * 100:.2f}%",
            f"{(1 - np.min(benchmark_cumulative_returns) / np.max(benchmark_cumulative_returns[0:np.argmin(benchmark_cumulative_returns)])) * 100:.2f}%",
            f"{benchmark_annual_return / benchmark_annual_volatility:.2f}"
        ]
    }
    
    metrics_df = pd.DataFrame(metrics)
    display(metrics_df.style.set_caption("Performance Metrics (Simulated)"))
else:
    print("No portfolio available for performance simulation.")

## Conclusion

In this notebook, we've demonstrated the key features of the WorldScreener package:

1. **Stock Screening**: Finding value stocks across different regions using customizable criteria
2. **Detailed Analysis**: Performing in-depth analysis of individual stocks
3. **Portfolio Creation**: Building a diversified portfolio of value stocks
4. **Visualization**: Creating informative charts and reports
5. **Performance Simulation**: Simulating potential portfolio performance

The WorldScreener package provides a comprehensive toolkit for value investors to identify, analyze, and track investment opportunities across global markets.